# Latent Net implementation

>> Notes to the new implementation - debugging


>> - the implementation of the Fourier features is wrong. get back to that later
>> - make sure that the train function loops over the parameters that are part  of the train set (i.e. the alphas that define the training sims)
>> - the append operation
>> - plot the latent state
>> - remove the train_snaphots and test_snapshots variables
>> - for each parameter, I train always on the same positions. to be changed?

In [1]:
# Install PyTorch
try:
  import torch
except ImportError:
  !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  import torch

#torch.set_default_dtype(torch.float32)

In [2]:
# Install PyG
try:
  import torch_geometric
except ImportError:
  !pip3 install torch_geometric
  import torch_geometric

In [3]:
import numpy as np

In [4]:
# Import some utils from gca-rom
import sys
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/Fra-Sala/gnn_time.git
    sys.path.append('gnn_time')
else:
    sys.path.append('./..')
    
from latent_net import dynamics_network, initialization, loader, preprocessing_scale, train

In [5]:
variable = 'U'
problem_name = 'lid_cavity'
print("\nProblem: ", problem_name)
print("Variable: ", variable)



Problem:  lid_cavity
Variable:  U


## Set training parameters

In [6]:
train_rate = 50 # percentage of data used for training
dim_latent = 10
epochs = 1000
step_integration = 1e-1   # time step forward euler
pos_per_batch = 20
num_batches = 20
# scaling_type = 2 
# scaler_number = 3
final_time_simulation = 2.0 # do not change even if you consider less snapshots (shorter simulations)
argv = [problem_name, variable, train_rate, dim_latent, step_integration, epochs, pos_per_batch, num_batches, final_time_simulation]
HyperParams = dynamics_network.HyperParams(argv)

# Initialization

In [7]:
device = initialization.set_device()
initialization.set_reproducibility(HyperParams)

Device used:  cpu


# Load dataset

In [8]:
if 'google.colab' in str(get_ipython()):
    dataset_dir = '/content/gnn_time/dataset/'+problem_name+'_unstructured.mat'
else:
    dataset_dir = '../dataset/'+problem_name+'_unstructured.mat'

# Import the data from the mat file 
dataset = loader.LoadDataset(dataset_dir, variable)

# Delete the initial condition of each case
dataset.U = np.delete(dataset.U, np.s_[::10], 1)

# Take first 3 snaps for two simulations
# cols_to_delete = list(range(3, 9)) + list(range(12, dataset.U.shape[1]))
# dataset.U = np.delete(dataset.U, cols_to_delete, 1)


### DEBUG: take only the first two snapshots

#dataset.U = np.delete(dataset.U,np.s_[1:],1 )

In [9]:
dataset.U

tensor([[3.7960e-02, 6.1421e-02, 6.1421e-02,  ..., 1.2535e+00, 1.2535e+00,
         7.7471e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [9.2720e-04, 1.9940e-03, 2.6758e-03,  ..., 1.8098e-02, 3.5898e-02,
         3.8393e-02],
        [9.4044e-04, 2.0228e-03, 2.7146e-03,  ..., 1.8509e-02, 3.6612e-02,
         3.9172e-02],
        [4.1225e-03, 7.8296e-03, 9.0348e-03,  ..., 2.1059e-01, 2.1317e-01,
         1.6319e-01]])

In [10]:
# if 'google.colab' in str(get_ipython()):
#     u_t_latent = np.load('/content/gnn_time/lid_driven_cavity_fenics/u_t_latent.npy')
# else:
#     u_t_latent = np.load('../lid_driven_cavity_fenics/u_t_latent.npy')

# Import the single Fourier mode
if 'google.colab' in str(get_ipython()):
    params = np.load('/content/gnn_time/lid_driven_cavity_fenics/alpha1.npy')
else:
    params = np.load('../lid_driven_cavity_fenics/alpha1.npy')

############ TAKE ONLY THE FIRST two PARAMETER (FOURIER COEFF) #############

time = np.linspace(0.0, 1.8, 10)

# Get rid of the initial condition evaluation (0.0)
# u_t_latent = np.delete(u_t_latent, np.s_[::10], 0)   
# u_t_tensor = torch.from_numpy(u_t_latent)
time_tensor = torch.from_numpy(time)
print("Number of simulations (series of snaps):", len(params))
print("Number of instants of time:", len(time)-1)

#alpha1_vec = alpha1_vec.reshape(-1,1)


Number of simulations (series of snaps): 20
Number of instants of time: 9


In [11]:
train_loader, test_loader, scaler_all,\
scaler_test, VAR_all, VAR_test, train_snapshots,\
test_snapshots, position_dataset, params_train, params_test = preprocessing_scale.process_and_scale_dataset(dataset, HyperParams, params)

Number of nodes processed:  15681
Number of simulations processed:  20


In [12]:
data_test_iterator = iter(test_loader)
data_test = next(data_test_iterator).to(device)
data_test[:, :, 0]

tensor([[ 1.0000, -1.0000, -1.0000,  ..., -0.9500, -0.9492, -0.7889],
        [ 1.0000, -1.0000, -1.0000,  ..., -0.9306, -0.9293, -0.7689],
        [ 1.0000, -1.0000, -1.0000,  ..., -0.8994, -0.8973, -0.7448],
        ...,
        [ 1.0000, -1.0000, -1.0000,  ..., -0.9711, -0.9705, -0.6640],
        [ 1.0000, -1.0000, -1.0000,  ..., -0.9427, -0.9416, -0.6599],
        [ 1.0000, -1.0000, -1.0000,  ..., -0.9009, -0.8989, -0.5787]])

# Define the architecture

In [13]:
# The dynamics net takes u(t) as input and s(t)
dyn_input_size = 1 + HyperParams.dim_latent
dyn_hidden_size = 9
dim = 2 # we are in 2D

#rec_input_size = dim*HyperParams.rff_encoded_mult*HyperParams.batch_pos_size + HyperParams.dim_latent
rec_input_size = dim*HyperParams.batch_pos_size + HyperParams.dim_latent
rec_hidden_size = dyn_hidden_size
rec_output_size = HyperParams.batch_pos_size

dyn_model = dynamics_network.DynNet(dyn_input_size, dyn_hidden_size, HyperParams.dim_latent)
rec_model = dynamics_network.RecNet(rec_input_size, rec_hidden_size, rec_output_size)
dyn_model = dyn_model.to(device)
rec_model = rec_model.to(device)

torch.set_default_dtype(torch.float32)
optimizer = torch.optim.Adam([
    {'params': dyn_model.parameters()},
    {'params': rec_model.parameters()}
], lr=HyperParams.learning_rate,  weight_decay=HyperParams.weight_decay)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=HyperParams.miles, gamma=HyperParams.gamma)

# Train the network

In [14]:
train_history, test_history = train.train_dyn_rec_nets(dyn_model, rec_model, optimizer, scheduler,\
                          device, params_train, params_test, time_tensor, train_loader, test_loader, position_dataset,\
                              train_snapshots, test_snapshots, HyperParams)

  2%|▏         | 15/1000 [00:10<11:52,  1.38it/s, Loss(training)=0.31, Loss(test)=0.296] 

In [ ]:
from matplotlib import pyplot as plt

# semilogy

plt.semilogy(train_history['loss'], label='train loss')
plt.semilogy(test_history['loss'], label='test loss')
plt.legend()

plt.grid()
#finer grid
plt.minorticks_on()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()
